# Geospatial Site Competitiveness Scoring Code


1) Train a **RandomForest classifier** to predict **revenue tier** using geospatial features: `population_density`, `bus_stop_density`, `median_income`, `breakfast_shop_density`.
2) Evaluate accuracy on a holdout set and run an **ablation** by dropping `bus_stop_density`.
3) Use the model's **P(high revenue)** as an *expected revenue signal*.
4) Build a **cost proxy** using `expected_rent`, standardized **within locality**.
5) Compute a composite: **z(expected_revenue) − z(expected_rent_by_locality)**.
6) Map to **0–5** as the final competitiveness score.

## Setup

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
np.random.seed(42)
pd.set_option('display.max_columns', 50)

## 1) Create a synthetic dataset

In [ ]:
n = 1200
localities = [f'L{i:02d}' for i in range(1, 9)]
loc = np.random.choice(localities, size=n, p=np.random.dirichlet(np.ones(len(localities))))

loc_pop_boost = {l: np.random.uniform(-0.5, 0.5) for l in localities}
loc_rent_base = {l: np.random.uniform(20, 60) for l in localities}

population_density = np.random.gamma(shape=8, scale=500, size=n)
bus_stop_density = np.random.gamma(shape=6, scale=2, size=n)
median_income = np.random.normal(45, 8, size=n)
breakfast_shop_density = np.random.gamma(shape=5, scale=1.2, size=n)

expected_rent = np.array([loc_rent_base[l] for l in loc]) + 0.005 * population_density \
                 + 0.2 * bus_stop_density + np.random.normal(0, 2, size=n)

latent = (0.004 * population_density + 0.03 * median_income
          + 0.2 * np.log1p(breakfast_shop_density) + 0.03 * np.log1p(bus_stop_density)
          + np.array([loc_pop_boost[l] for l in loc]) + np.random.normal(0, 0.5, size=n))

q1, q2 = np.quantile(latent, [0.4, 0.75])
revenue_tier = np.digitize(latent, bins=[q1, q2])

df = pd.DataFrame({
    'locality': loc,
    'population_density': population_density,
    'bus_stop_density': bus_stop_density,
    'median_income': median_income,
    'breakfast_shop_density': breakfast_shop_density,
    'expected_rent': expected_rent,
    'revenue_tier': revenue_tier
})
df.head()

## 2) Train a RandomForest revenue-tier model

In [ ]:
FEATURES = ['population_density','bus_stop_density','median_income','breakfast_shop_density']
X = df[FEATURES]
y = df['revenue_tier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Accuracy (with bus_stop_density):', round(acc, 3))
print('\nClassification report:')
print(classification_report(y_test, y_pred, digits=3))

### Feature importance

In [ ]:
importances = rf.feature_importances_
fig = plt.figure()
plt.bar(range(len(FEATURES)), importances)
plt.xticks(range(len(FEATURES)), FEATURES, rotation=20)
plt.title('RandomForest feature importances')
plt.tight_layout()
plt.show()

## 3) Ablation: drop `bus_stop_density`

In [ ]:
FEATURES_ABL = ['population_density','median_income','breakfast_shop_density']
X2 = df[FEATURES_ABL]
from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.25, stratify=y, random_state=42)
rf2 = RandomForestClassifier(n_estimators=300, random_state=42)
rf2.fit(X2_train, y2_train)
y2_pred = rf2.predict(X2_test)
from sklearn.metrics import accuracy_score
acc2 = accuracy_score(y2_test, y2_pred)
print('Accuracy (WITHOUT bus_stop_density):', round(acc2, 3))
print('Δ accuracy:', round(acc2 - acc, 3))

## 4) Expected revenue signal & rent standardization by locality

In [ ]:
p_high = rf2.predict_proba(df[FEATURES_ABL])[:, 2]
z_rev = (p_high - p_high.mean()) / (p_high.std() + 1e-9)
df['_rent_mean_by_loc'] = df.groupby('locality')['expected_rent'].transform('mean')
df['_rent_std_by_loc'] = df.groupby('locality')['expected_rent'].transform('std').fillna(1.0)
z_rent_by_loc = (df['expected_rent'] - df['_rent_mean_by_loc']) / (df['_rent_std_by_loc'] + 1e-9)
df['z_expected_revenue'] = z_rev
df['z_expected_rent_by_loc'] = z_rent_by_loc
df[['locality','expected_rent','z_expected_rent_by_loc','z_expected_revenue']].head()

## 5) Composite score and 0–5 mapping

In [ ]:
df['composite_score'] = df['z_expected_revenue'] - df['z_expected_rent_by_loc']
bins = pd.qcut(df['composite_score'], q=6, labels=False, duplicates='drop')
df['competitiveness_0_5'] = bins
df[['locality','composite_score','competitiveness_0_5']].head()

## 6) Save results

In [ ]:
out = df.copy()
out['p_high_revenue'] = rf2.predict_proba(df[FEATURES_ABL])[:,2]
keep_cols = ['locality','population_density','bus_stop_density','median_income','breakfast_shop_density',
             'expected_rent','revenue_tier','p_high_revenue','z_expected_revenue',
             'z_expected_rent_by_loc','composite_score','competitiveness_0_5']
out = out[keep_cols]
csv_path = '/mnt/data/site_competitiveness_demo.csv'
out.to_csv(csv_path, index=False)
print('Saved:', csv_path)